# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [53]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
df.dtypes

Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object

In [54]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
mean_df = df.groupby(['Cabin'])['Fare'].mean().reset_index()
max_df = df.groupby(['Cabin'])['Fare'].max().reset_index()
mode_df = df.groupby(['Cabin'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
min_df = df.groupby(['Cabin'])['Fare'].min().reset_index()
count_df = df.groupby(['Cabin'])['Fare'].count().reset_index()

df_temp = pd.merge(mean_df, max_df, how='left', on = ['Cabin'])
df_temp = pd.merge(df_temp, mode_df, how='left', on=['Cabin'])
df_temp = pd.merge(df_temp, min_df, how='left', on=['Cabin'])
df_temp = pd.merge(df_temp, count_df, how='left', on=['Cabin'])
df_temp.columns=['Cabin', 'Fare_mean', 'Fare_max', 'Fare_mode', 'Fare_min', 'Fare_count']
df_temp.head(20)

,Cabin,Fare_mean,Fare_max,Fare_mode,Fare_min,Fare_count
0,A10,40.1250,40.1250,40.1250,40.1250,1
1,A14,52.0000,52.0000,52.0000,52.0000,1
2,A16,39.6000,39.6000,39.6000,39.6000,1
3,A19,26.0000,26.0000,26.0000,26.0000,1
4,A20,56.9292,56.9292,56.9292,56.9292,1
5,A23,30.0000,30.0000,30.0000,30.0000,1
6,A24,50.4958,50.4958,50.4958,50.4958,1
7,A26,35.5000,35.5000,35.5000,35.5000,1
8,A31,31.0000,31.0000,31.0000,31.0000,1
9,A32,50.0000,50.0000,50.0000,50.0000,1


In [58]:
df_new = pd.merge(df, df_temp, how='left', on=['Cabin'])

In [59]:
df_new

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_mean,Fare_max,Fare_mode,Fare_min,Fare_count
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,NaN,NaN,NaN,NaN,NaN
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,71.2833,71.2833,71.2833,71.2833,1.0
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,NaN,NaN,NaN,NaN,NaN
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,53.1000,53.1000,53.1000,53.1000,2.0
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,NaN,NaN,NaN,NaN,NaN
887,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,30.0000,30.0000,30.0000,30.0000,1.0
888,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,NaN,NaN,NaN,NaN,NaN
889,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,30.0000,30.0000,30.0000,30.0000,1.0


In [61]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [62]:
# 原始特徵 + 邏輯斯迴歸
"""
df_NewandOld = pd.merge()
"""
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.70041428661101

In [63]:
# 新特徵 + 邏輯斯迴歸
num_features = []
for dtype, feature in zip(df_new.dtypes, df_new.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df_new_number = df_new[num_features]
df_new_number = df_new_number.fillna(-1)
MMEncoder = MinMaxScaler()
df_new_number.head()

train_X = MMEncoder.fit_transform(df_new_number)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

10 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_mean', 'Fare_max', 'Fare_mode', 'Fare_min', 'Fare_count']



0.6970309459544286

In [69]:
df_new_number_only = df_new.loc[:,['Fare_mean', 'Fare_max', 'Fare_mode', 'Fare_min', 'Fare_count']]
df_new_number_only = df_new_number_only.fillna(-1)
MMEncoder = MinMaxScaler()
df_new_number.head()

train_X = MMEncoder.fit_transform(df_new_number_only)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6936413282279832

In [68]:
df_new.loc[:,['Cabin', 'Fare_mean', 'Fare_max', 'Fare_mode', 'Fare_min', 'Fare_count']]

,Cabin,Fare_mean,Fare_max,Fare_mode,Fare_min,Fare_count
0,NaN,NaN,NaN,NaN,NaN,NaN
1,C85,71.2833,71.2833,71.2833,71.2833,1.0
2,NaN,NaN,NaN,NaN,NaN,NaN
3,C123,53.1000,53.1000,53.1000,53.1000,2.0
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
886,NaN,NaN,NaN,NaN,NaN,NaN
887,B42,30.0000,30.0000,30.0000,30.0000,1.0
888,NaN,NaN,NaN,NaN,NaN,NaN
889,C148,30.0000,30.0000,30.0000,30.0000,1.0
